In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.user

UserSettings

id: 06641123-e808-793b-8000-f349a1d489b2
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'alpha_vantage_api_key': SecretStr('**********'), 'fmp_api_key': SecretStr('**********'), 'intrinio_api_key': None, 'tradier_api_key': None, 'nasdaq_api_key': None, 'benzinga_api_key': None, 'tiingo_token': SecretStr('**********'), 'biztoc_api_key': SecretStr('**********'), 'tradier_account_type': None, 'tradingeconomics_api_key': None, 'polygon_api_key': SecretStr('**********'), 'fred_api_key': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [6]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes
obb.account.refresh()

# Logout
# obb.account.logout()

In [8]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [9]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [10]:
symbol = "MRK"
company = "Merck"

In [11]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 06641123-ee8b-735f-8000-8df9782bc909
results: [{'symbol': 'MRK', 'one_day': -0.0013, 'wtd': None, 'one_week': 0.02, 'mtd...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [12]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=-0.0013, wtd=None, one_week=0.02, mtd=None, one_month=0.031, qtd=None, three_month=0.036699999999999997, six_month=0.2458, ytd=0.193, one_year=0.10310000000000001, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.013999999999999999, volatility_month=0.0148, price=130.06, volume=5722074.0, average_volume=8210000.000000001, relative_volume=0.7, analyst_recommendation=None, analyst_score=1.52)]

In [13]:
obj.model_dump_json()


'{"id":"06641123-ee8b-735f-8000-8df9782bc909","results":[{"symbol":"MRK","one_day":-0.0013,"wtd":null,"one_week":0.02,"mtd":null,"one_month":0.031,"qtd":null,"three_month":0.036699999999999997,"six_month":0.2458,"ytd":0.193,"one_year":0.10310000000000001,"two_year":null,"three_year":null,"four_year":null,"five_year":null,"ten_year":null,"max":null,"volatility_week":0.013999999999999999,"volatility_month":0.0148,"price":130.06,"volume":5722074.0,"average_volume":8210000.000000001,"relative_volume":0.7,"analyst_recommendation":null,"analyst_score":1.52}],"provider":"finviz","warnings":null,"chart":null,"extra":{"metadata":{"arguments":{"provider_choices":{"provider":"finviz"},"standard_params":{"symbol":"MRK"},"extra_params":{}},"duration":167874416,"route":"/equity/price/performance","timestamp":"2024-05-12T15:02:22.741924"}}}'

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 129.9,
   'bid_size': 800,
   'ask': 130.0,
   'ask_size': 800,
   'last_price': 130.06,
   'open': 130.82,
   'high': 130.88,
   'low': 129.97,
   'volume': 5722627,
   'prev_close': 130.23,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 126.5012,
   'ma_200d': 114.20525,
   'volume_average': 8215488.0,
   'volume_average_10d': 6810530.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 164458750,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-12T15:02:22.947629'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-12,118.00,118.550,116.695,117.14,5233088.0,117.2581,54248
2023-05-15,117.14,117.740,115.490,116.37,5697236.0,116.3517,70438
2023-05-16,115.88,116.760,115.290,116.08,4278281.0,116.2071,60730
2023-05-17,116.37,116.655,113.480,114.76,7210383.0,114.5498,78904
2023-05-18,114.14,115.070,113.330,114.00,6437144.0,114.0864,75952
...,...,...,...,...,...,...,...
2024-05-06,127.18,127.730,126.760,127.57,6419935.0,127.4189,80172
2024-05-07,127.10,130.425,127.070,130.38,6415640.0,129.3458,87045
2024-05-08,130.58,131.510,129.330,129.55,6580073.0,129.7913,74021


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-12,411.59,413.6400,409.070,413.42,501017,70481548.0,411.1564
2023-05-15,413.01,413.4300,410.230,412.22,427865,54289383.0,412.1201
2023-05-16,410.25,412.8150,410.240,411.86,447536,57705495.0,411.3774
2023-05-17,415.23,415.8550,410.635,412.35,585038,86786957.0,413.4302
2023-05-18,419.23,419.6700,414.670,414.90,672492,97177195.0,417.1828
...,...,...,...,...,...,...,...
2024-05-06,516.57,516.6100,513.300,513.75,444747,47236717.0,514.6357
2024-05-07,517.14,518.5700,516.450,517.56,432286,50977654.0,517.4968
2024-05-08,517.19,517.7400,515.140,515.26,369150,42012599.0,516.9211


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.62,2024-05-10 20:00:01+00:00,56.35,56.8000,56.290,3927323,56.13,...,66.15,54.12,58.7556,60.24095,3127003,1.003363e+11,1772100000,4.61,12.28,2024-05-23 12:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,76.11,2024-05-10 20:00:02+00:00,75.80,76.2500,75.530,5103040,75.44,...,77.05,48.66,71.6642,63.13665,7082303,1.351759e+11,1776060000,2.39,31.85,2024-07-16 04:00:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,198.77,2024-05-10 20:00:02+00:00,198.54,199.3399,198.270,7529790,197.50,...,200.94,133.96,192.1660,166.00345,8842138,5.708018e+11,2871670000,16.58,11.99,2024-07-12 04:00:00+00:00
3,MS,Morgan Stanley,NYSE,98.28,2024-05-10 20:00:02+00:00,98.74,98.9900,97.625,6118425,98.11,...,98.99,69.42,91.2008,85.73105,8073372,1.597207e+11,1625160000,5.50,17.87,2024-07-16 04:00:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,tags,id,amp_url,publisher
date,,,,,,,,,,
2024-05-07 10:20:07+00:00,Should You Invest in the Invesco S&P 500 Equal...,Sector ETF report for RSPH,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2269228/shoul...,"RSPH,MRK,CI,WST,XLV,VHT",Zacks Equity Research,None,fsu82w8dXastE38DzeowYOxzq9FCS7S3msQLXKUJ6YY,https://www.zacks.com/amp/stock/news/2269228/s...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-07 15:30:00+00:00,A Look at Pharma ETFs Post Q1 Earnings,"Many industry bigwigs reported solid results, ...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2269782/a-loo...,"BMY,JNJ,MRK,LLY,PJP,IHE,XPH,PPH,FTXH",Sweta Killa,None,sNchkXpEjItDlY19xKtiys_FBgeVOIoZIEhqQEd_o3g,https://www.zacks.com/amp/stock/news/2269782/a...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-08 06:26:57+00:00,Jim Cramer Advises Investors To Brace For Econ...,"Jim Cramer, the host of CNBC’s “Mad Money,” ha...",[{'url': 'https://cdn.benzinga.com/files/image...,https://www.benzinga.com/news/24/05/38691510/j...,"AAPL,BLDR,GOOG,NVDA,MRK,PFE,GOOGL,META",Benzinga Neuro,"News,Global,Economics,Federal Reserve,Markets",nabJYp9Ewiso_ihbgnuIkdcXtn3Ihy-oucZmO2ALnCk,https://www.benzinga.com/amp/content/38691510,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-09 13:00:16+00:00,"Investors Heavily Search Merck & Co., Inc. (MR...","Recently, Zacks.com users have been paying clo...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2271265/inves...,MRK,Zacks Equity Research,None,LThWA5Sh3t7k8BJ8547R71qJY_9Mgkha7WMQAbZTGro,https://www.zacks.com/amp/stock/news/2271265/i...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-10 15:18:00+00:00,Pharma Stock Roundup: PFE DMD Study Patient De...,Pfizer (PFE) reports the death of a participan...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272311/pharm...,"PFE,MRK,LLY",Kinjel Shah,None,LuH9atBP810MgCmcYvB9qbOEDI5RlsMVZB1l1s1W8D8,https://www.zacks.com/amp/stock/news/2272311/p...,{'favicon_url': 'https://s3.polygon.io/public/...


# Question Answering Agent

In [21]:
MODEL = "gpt-4-turbo"

MAX_INPUT_TOKENS = 65536     # includes text of all headlines
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0



In [22]:
tools = {}
tools_dict = {}
tools_examples = {}

In [23]:
def get_10k_item1_from_symbol(symbol):
    """
    Gets item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    return item1_text


tools["get_10k_item1_from_symbol"] = {
    "type": "function",
    "function": {
        "name": "get_10k_item1_from_symbol",
        "description": "Retrieves item 1 of the latest 10-K annual report filing for a given symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The symbol to get the 10-K item 1 for"
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_10k_item1_from_symbol"] = get_10k_item1_from_symbol

item_1_text = get_10k_item1_from_symbol(symbol)
len(item_1_text)


190265

In [24]:
system_prompt = """
Role: You are an AI stock market assistant tasked with providing up-to-date and detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate and concise answers relevant to their questions about individual stocks.

Capabilities: You are given a number of tools as functions. Use as many tools as needed to ensure all information provided is timely, accurate, concise, relevant, and specific to the user's query.

Instructions: First, analyze the query to fully understand the intent of the user, what data they require, and which tool will satisfy their requirement.
Then, respond to the investor queries with relevant data and descriptive text, using the current data from the most relevant tools.
If the question is not about a specific stock, if no relevant tool is available, or if uncertain or lacking data, say so clearly and suggest alternative resources or data sources that may help.

Example Interaction:
Investor: What is the current P/E ratio of Apple Inc.?
AI Response: As of the close of trading on March 10, 2024, Apple Inc.'s P/E ratio is 28.45.
"""


In [25]:
def eval_tool(tool_call, verbose=False):
    """
    Given an OpenAI tool_call response,
    evaluates the tool function using the arguments provided by OpenAI,
    and returns the message to send back to OpenAI, including the function return value.

    Args:
        tool_call (object): The OpenAI tool_call response.

    Returns:
        dict: The message to send back to OpenAI, containing the tool_call_id, role, name, and value returned by the tool call.

    """
    function_name = tool_call.function.name
    # look up the function based on the name
    fn = tools_dict[function_name]
    # make the tool call's json args into a dict
    kwargs = json.loads(tool_call.function.arguments)
    if verbose:
        print(f"{function_name}({str(kwargs)})")
    # call function with the args and return value
    fn_value = fn(**kwargs)
    if verbose:
        output = str(fn_value)
        if len(output) > 1000:
            output=output[:1000] + "..."
        print(output)

    return {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": tool_call.function.name,
        "content": fn_value,
    }


def get_response(messages, json_format=False):
    """
    Get a single response from ChatGPT based on a chain of messages.

    Args:
        messages (list): A list of message objects representing the conversation history.
        json_format(boolean): True if JSON response requested. (Last message must express the request for JSON response.)

    Returns:
        dict: A response object containing the generated response from ChatGPT.

    Raises:
        OpenAIError: If there is an error during the API call.

    Example:
        >>> messages = [
        ...     {"role": "system", "content": "You are a helpful assistant."},
        ...     {"role": "user", "content": "What's the weather like today?"},
        ... ]
        >>> response = get_response(messages)
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=list(tools.values()),
        tool_choice="auto",  # auto is default, but we'll be explicit
        response_format={"type": "json_object"} if json_format else None,
    )

    return response


def get_response_and_eval(messages, json_format=False, verbose=False):
    """
    Sends a list of messages to OpenAI and returns the response.
    If tool calls are returned, calls all the tools and sends the values back to OpenAI.
    If further tool calls returned, iterates until no more tool calls are returned and
    'stop' is returned as finish_reason, then returns the response.

    Args:
        messages (list): A list of messages to send to OpenAI.
        json_format (boolean): If the final response should be in JSON format.
        verbose (bool, optional): If True, prints additional information. Defaults to False.

    Returns:
        response: The final response object returned by OpenAI.

    Raises:
        None

    """
    response = get_response(messages, json_format=json_format)
    choice = response.choices[0]
    response_message = choice.message
    finish_reason = choice.finish_reason

    if verbose:
        print(choice)

    while finish_reason != 'stop':
        # Extend conversation with assistant's reply
        messages.append(response_message)
        if finish_reason == 'tool_calls':
            tool_calls = response_message.tool_calls
            if verbose:
                print(tool_calls)
            # Call the tools and add all return values as messages
            for tool in tool_calls:
                messages.append(eval_tool(tool, verbose=verbose))
            # Get next response
            response = get_response(messages, json_format=json_format)
            choice = response.choices[0]
            response_message = choice.message
            finish_reason = choice.finish_reason
            if verbose:
                output = str(choice)
                output = output[:1000] + "..." if len(output) > 1000 else output
                print(output)
        else:
            print('finish_reason: ', finish_reason)
            break

    return response


def agent_query(user_message, verbose=False):
    """
    Sends a user message to OpenAI and retrieves the response, calling tools as necessary.

    Args:
        user_message (str): The message from the user.
        verbose (bool, optional): Display intermediate tool calls and return values. Defaults to False.

    Returns:
        str: The response from the agent.


    Example:
        >>> agent_query("Hello")
        'Hello! How can I assist you today?'
    """

    # add descriptions of available tools in system prompt
    tempstr = ""
    for t in tools.values():
        tname = t['function']['name']
        tdesc = t['function']['description']
        texample = tools_examples.get(tname)
        tempstr += f"{tname} : {tdesc}"
        if texample:
            if type(texample) is list:
                tex_str = str(texample[-4:])
            else:
                tex_str = str(texample)
            tempstr += f" Example Output: {tex_str} "
        tempstr += "\n---\n\n"
    # tool_descs = "\n".join([f"{tool['function']['name']} : {tool['function']['description']}" for tool in tools.values()])
    tool_descs = tempstr
    current_system_prompt = system_prompt + f"""

Available tools delimited by ---:
{tool_descs}
    """
    # print(current_system_prompt)

    messages = [{"role": "system", "content": current_system_prompt},
                {"role": "user", "content": user_message}]
    response = get_response_and_eval(messages, json_format=False, verbose=verbose)
    display(Markdown(response.choices[0].message.content))



In [26]:
client = OpenAI()
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(messages)
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

display(Markdown(response_str))


The question "What is the airspeed velocity of an unladen swallow?" is famously known from the movie *Monty Python and the Holy Grail*. In the movie, it's used as a humorous question without a straightforward answer.

However, if you are looking for actual bird flight data, European Swallows (which are presumably the subject of the Monty Python joke) are estimated to have an average cruising airspeed of roughly 11 meters per second, or about 24 miles per hour.

In [27]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(user_message, verbose=False)


The latest annual report for Microsoft Corporation (Item 1 from Form 10-K) provides an in-depth overview of the company's business, focusing especially on their shift towards infusing artificial intelligence across their portfolio. Here are the main points summarized:

1. **Business Overview**: Microsoft positions itself as a comprehensive technology company aimed at empowering every individual and organization globally to achieve more. Leveraging AI, Microsoft focuses on creating innovative solutions that enhance productivity, improve outcomes in education and health, and increase public-sector efficiencies.

2. **Product and Service Offerings**:
   - **Software and Devices**: Microsoft develops and supports software, services, devices, and solutions. This includes cloud solutions, productivity applications, business solutions applications, and devices like PCs, tablets, and gaming consoles.
   - **AI Innovations**: The company is intensively investing in AI to integrate generative AI capabilities across consumer and commercial offerings, enhancing tools such as Microsoft Teams, Outlook, and the Cloud suite.

3. **Strategic Ambitions**: Microsoft has outlined its ambitions to (i) reinvent productivity and business processes, (ii) build intelligent cloud and edge computing platforms, and (iii) create more personalized computing experiences.

4. **Research and Development**: The emphasis is on continuous innovation, particularly in AI to foster productivity and collaboration applications. Microsoft 365, Teams, Dynamics 365, and LinkedIn are central to these efforts.

5. **Competitive Landscape**: Microsoft recognizes intense competition from other technology firms, including traditional software, hardware vendors, and newer AI-centric enterprises. The company believes its integrated products and cloud-based solutions offer a competitive advantage.

6. **Corporate Responsibility and Social Impact**:
   - **Sustainability**: Microsoft is committed to sustainability, aiming for carbon negativity and zero waste by 2030.
   - **Digital Skills and Equity**: Expanding initiatives like digital skills training and broader inclusivity in technology accessibility.
   - **Data Governance and Security**: Microsoft stresses responsible AI use, emphasizing transparency, security, and compliance to foster user trust.

7. **Future Outlook**: Microsoft aims to leverage its cloud infrastructure to explore new market opportunities, including mixed reality and quantum computing innovations, focusing on the long-term expansion and transformation of business and societal operations driven by AI and digital technologies.

This summary provides an overview of Microsoft's business operations, strategic direction, and commitment to leveraging advanced technology for growth and societal impact, aligning with their core mission of empowerment and innovation.

In [28]:
# query search using Sec-api
# queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])

# query = {
#   "query": "ticker:TSLA AND filedAt:[2024-01-01 TO 2024-12-31] AND formType:\"10-K\"",
#   "from": "0",
#   "size": "100",
#   "sort": [{ "filedAt": { "order": "desc" } }]
# }

# response = queryApi.get_filings(query)

# print(response)
# extractorApi = ExtractorApi(os.getenv('SEC_API_KEY'))
# url_10k = 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm'
# item_1_text = extractorApi.get_section(url_10k, "1", "text")
# print(item_1_text)

# 10-K example
# url_10k = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20231231.htm"

# item_1A_text = extractorApi.get_section(url_10k, "1A", "text")
# item_1_text = extractorApi.get_section(url_10k, "1", "text")

# item_7_html = extractorApi.get_section(url_10k, "7", "html")

# 10-Q example
# url_10q = "https://www.sec.gov/Archives/edgar/data/1318605/000095017022006034/tsla-20220331.htm"

# part2_item_1A_text = extractorApi.get_section(url_10q, "part2item1a", "text")

# 8-K example
# url_8k = "https://www.sec.gov/Archives/edgar/data/66600/000149315222016468/form8-k.htm"



# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [29]:
# TODO:
# fix historical functions with dates, simplify output

# check additional prompt improvements

# check all results , see if correct

# use assistants API


In [30]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [31]:
# using the openapi.json swagger definition , output a function to call the openbb function
# might be a better way to do this with standard openapi tools to parse the spec, but didn't find one.

def fn_str(d):
    operationId = d['get']['operationId']
    operation = operationId.replace('_', '.')
    docstring = f"{d['get']['description']}"
    req_params = ", ".join([p['name'] for p in d['get']['parameters'] if p['required']])

    retval = f"""
def get_{operationId}({req_params}):
    \"\"\"{docstring}
    \"\"\"

    retval = None
    try:
        obj = obb.{operation}({req_params})
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval

"""
    return retval

path_str = '/api/v1/equity/search'
fn_json = data['paths'][path_str]
fn_json
print(fn_str(fn_json))


def get_equity_search(provider):
    """Search for stock symbol, CIK, LEI, or company name.
    """

    retval = None
    try:
        obj = obb.equity.search(provider)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval




In [32]:
# output all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)
        req_params = ", ".join([p['name'] for p in fn_json['get']['parameters'] if p['required']])
        opt_params = ", ".join([p['name'] for p in fn_json['get']['parameters'] if not p['required']])

        operationId = fn_json['get']['operationId']
        operation = operationId.replace('_', '.')
        data['paths']

        print(f"operation:            {operation}")
        print(f"operationId:          {operationId}")
        print(f"summary:              {fn_json['get']['summary']}")
        print(f"description:          {fn_json['get']['description']}")
        print(f"required parameters:  {req_params}")
        print(f"optional parameters:  {opt_params}")

        print(fn_str(fn_json))

        print()


/api/v1/equity/calendar/ipo
operation:            equity.calendar.ipo
operationId:          equity_calendar_ipo
summary:              Ipo
description:          Get historical and upcoming initial public offerings (IPOs).
required parameters:  provider
optional parameters:  symbol, start_date, end_date, limit, status, offer_amount_greater_than, offer_amount_less_than, is_spo

def get_equity_calendar_ipo(provider):
    """Get historical and upcoming initial public offerings (IPOs).
    """

    retval = None
    try:
        obj = obb.equity.calendar.ipo(provider)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval



/api/v1/equity/calendar/dividend
operation:            equity.calendar.dividend
operationId:          equity_calendar_dividend
summary:              Dividend
description:          Get historical and upcoming dividend payments. Includes dividend amount, ex-dividend and payment dates.
required parameters:  provid

In [33]:
def get_equity_search_symbol(search_str):
    """Given a search string, find the stock symbol of the top company whose name best matches the search string.
    """
    retval = None
    try:
        obj = obb.equity.search(search_str)
        retval = obj.results[0].symbol
    except Exception as exc:
        print(exc)
    return retval


tools["get_equity_search_symbol"] = {
    "type": "function",
    "function": {
        "name": "get_equity_search_symbol",
        "description": "Given a search string, find the stock symbol of the top company whose name best matches the search string.",
        "parameters": {
            "type": "object",
            "properties": {
                "search_str": {
                    "type": "string",
                    "description": "The search string."
                }
            },
            "required": ["search_str"]
        }
    }
}
tools_dict["get_equity_search_symbol"] = get_equity_search_symbol



In [34]:
output = get_equity_search_symbol(search_str='Broadcom')
tools_examples['get_equity_search_symbol'] = output
output


'AVGO'

In [35]:
agent_query("What is the stock symbol for Broadcom?")


The stock symbol for Broadcom is AVGO.

In [36]:
def get_quote_json(symbol):
    """Given a stock symbol, get the latest market data quote for the stock in JSON format.

    Args:
        symbol (str): The symbol of the stock.

    Returns:
        str: The JSON representation of the latest quote data for the given stock.

    Raises:f
        None

    Example:
        >>> get_quote_json('AAPL')
        '{"symbol": "AAPL", "price": 150.25, "volume": 1000, ...}'
    """
    retval = None
    retlist = obb.equity.price.quote(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


tools["get_quote_json"] = {
    "type": "function",
    "function": {
        "name": "get_quote_json",
        "description": "Given a stock symbol, get the latest market data quote for the stock in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_quote_json"] = get_quote_json

quote = get_quote_json(symbol)
example_output = json.loads(quote)
tools_examples["get_quote_json"]=example_output
example_output

{'symbol': 'MRK',
 'asset_type': 'stock',
 'name': 'MERCK & CO INC COM',
 'exchange': None,
 'bid': 129.9,
 'bid_size': 1,
 'bid_exchange': None,
 'ask': 130.0,
 'ask_size': 2,
 'ask_exchange': None,
 'quote_conditions': None,
 'quote_indicators': None,
 'sales_conditions': None,
 'sequence_number': None,
 'market_center': None,
 'participant_timestamp': None,
 'trf_timestamp': None,
 'sip_timestamp': None,
 'last_price': 129.49,
 'last_tick': 'up',
 'last_size': None,
 'last_timestamp': '2024-05-10T15:59:59',
 'open': 130.73,
 'high': 130.88,
 'low': 129.95,
 'close': 130.06,
 'volume': 5722089,
 'exchange_volume': None,
 'prev_close': 130.06,
 'change': -0.17,
 'change_percent': -0.001307,
 'year_high': 133.10000610351562,
 'year_low': 99.13999938964844,
 'iv30': 0.14267,
 'iv30_change': None,
 'iv30_change_percent': None,
 'iv30_annual_high': 0.25642999649047854,
 'hv30_annual_high': 0.20973699569702148,
 'iv30_annual_low': 0.1488199996948242,
 'hv30_annual_low': 0.11282099723815918

In [37]:
agent_query(f"What is the latest price for {symbol}")

As of the latest available data on May 10, 2024, the last trading price for Merck & Co Inc (MRK) was $129.49.

In [38]:
def get_company_profile_json(symbol):
    """
    Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format

    Parameters:
        symbol (str): The symbol of the company

    Returns:
        str: The JSON representation of the company's profile information
    """
    retval = None
    retlist = obb.equity.profile(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval

tools["get_company_profile_json"] = {
    "type": "function",
    "function": {
        "name": "get_company_profile_json",
        "description": "Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_company_profile_json"] = get_company_profile_json

profile_json = get_company_profile_json(symbol)
example_output = json.loads(profile_json)
tools_examples["get_company_profile_json"]=example_output
example_output


{'symbol': 'MRK',
 'name': 'Merck & Co Inc',
 'cik': None,
 'cusip': None,
 'isin': None,
 'lei': None,
 'legal_name': None,
 'stock_exchange': 'NYSE',
 'sic': None,
 'short_description': None,
 'long_description': 'Merck & Co., Inc. is a health care company, which engages in the provision of health solutions through its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. It operates through the following segments: Pharmaceutical, Animal Health, and Other. The Pharmaceutical segment includes human health pharmaceutical and vaccine products. The Animal Health segment discovers, develops, manufactures, and markets animal health products, such as pharmaceutical and vaccine products, for the prevention, treatment and control of disease in livestock, and companion animal species. The Other segment consists of sales for the non-reportable segments of healthcare services. The company was founded in 1891 and is headquartered in Rahway, NJ.',
 'ceo':

In [39]:
agent_query(f"Can you provide a basic company profile of {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_UPoI2HHx5hADCgUXRREy1RqM', function=Function(arguments='{"symbol":"MRK"}', name='get_company_profile_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_UPoI2HHx5hADCgUXRREy1RqM', function=Function(arguments='{"symbol":"MRK"}', name='get_company_profile_json'), type='function')]
get_company_profile_json({'symbol': 'MRK'})
{"symbol":"MRK","name":"Merck & Co Inc","cik":null,"cusip":null,"isin":null,"lei":null,"legal_name":null,"stock_exchange":"NYSE","sic":null,"short_description":null,"long_description":"Merck & Co., Inc. is a health care company, which engages in the provision of health solutions through its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. It operates through the following segments: Pharmaceutical, Animal Health, and 

Merck & Co., Inc. (symbol: MRK), traded on the NYSE, is a health care company primarily engaged in providing solutions through prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. Its operations are organized into three main segments: Pharmaceutical, Animal Health, and Other:

- **Pharmaceutical Segment**: Focuses on human health pharmaceutical and vaccine products.
- **Animal Health Segment**: Involves discovering, developing, manufacturing, and marketing pharmaceutical and vaccine products for livestock and companion animals.
- **Other Segment**: Includes sales from non-reportable healthcare services.

Founded in 1891, Merck is headquartered in Rahway, New Jersey, USA, and employs approximately 72,000 people. It is included in major indices such as the DJIA and S&P 500. The company has a market capitalization of approximately $329.42 billion with 2.53 billion shares outstanding and institutional ownership of about 78.09%. The company's next earnings date is scheduled for April 25, in the morning (BMO).

The company operates within the Healthcare sector and belongs to the 'Drug Manufacturers - General' industry category.

In [40]:
def get_equity_shorts_short_interest(symbol):
    """Given a stock symbol, get data on reported short volume and days to cover in JSON format.

    Args:
        symbol (str): The stock symbol for which to retrieve short interest data.

    Returns:
        str: The JSON-formatted data containing reported short volume and days to cover.

    Example:
        >>> get_equity_shorts_short_interest('AAPL')
        '{"short_volume": 10000, "days_to_cover": 5}'
    """
    retval = None
    try:
        obj = obb.equity.shorts.short_interest(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval


tools["get_equity_shorts_short_interest"] = {
    "type": "function",
    "function": {
        "name": "get_equity_shorts_short_interest",
        "description": "Given a stock symbol, get data on reported short volume and days to cover in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}

tools_dict["get_equity_shorts_short_interest"] = get_equity_shorts_short_interest

equity_shorts_short_interest = get_equity_shorts_short_interest(symbol)
example_output = json.loads(equity_shorts_short_interest)
tools_examples["get_equity_shorts_short_interest"]=example_output
example_output

{'settlement_date': '2021-07-15',
 'symbol': 'MRK',
 'issue_name': 'Merck & Co., Inc.',
 'market_class': 'NYSE',
 'current_short_position': 22245886.0,
 'previous_short_position': 23932756.0,
 'avg_daily_volume': 7804274.0,
 'days_to_cover': 2.85,
 'change': -1686870.0,
 'change_pct': -7.05}

In [41]:
agent_query(f"Provide latest statistics on short interest for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_S3bTbpoBPWi6z2FnV3xDxXGM', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_shorts_short_interest'), type='function')]))
[ChatCompletionMessageToolCall(id='call_S3bTbpoBPWi6z2FnV3xDxXGM', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_shorts_short_interest'), type='function')]
get_equity_shorts_short_interest({'symbol': 'MRK'})
{"settlement_date":"2021-07-15","symbol":"MRK","issue_name":"Merck & Co., Inc.","market_class":"NYSE","current_short_position":22245886.0,"previous_short_position":23932756.0,"avg_daily_volume":7804274.0,"days_to_cover":2.85,"change":-1686870.0,"change_pct":-7.05}
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The latest statistics on short interest for Merck & Co., Inc. (MRK) as of the settlemen

The latest statistics on short interest for Merck & Co., Inc. (MRK) as of the settlement date July 15, 2021, are as follows:
- **Current Short Position:** 22,245,886 shares
- **Previous Short Position:** 23,932,756 shares
- **Average Daily Volume:** 7,804,274 shares
- **Days to Cover:** 2.85 days
- **Change in Short Position:** -1,686,870 shares
- **Percentage Change:** -7.05%

This data indicates the number of shares that were sold short and not yet covered or closed out as of the latest reporting period. If you need more recent data or additional details, I recommend checking the most recent SEC filings or financial news updates directly related to MRK.

In [42]:
def get_equity_fundamental_historical_splits(symbol):
    """Given a stock symbol, get the company's historical stock splits in JSON format.

    Args:
        symbol (str): The stock symbol for the company.

    Returns:
        str: The JSON representation of the company's historical stock splits.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.historical_splits(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval


tools["get_equity_fundamental_historical_splits"] = {
    "type": "function",
    "function": {
        "name": "get_equity_fundamental_historical_splits",
        "description": "Given a stock symbol, get the company's historical stock splits in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}

tools_dict["get_equity_fundamental_historical_splits"] = get_equity_fundamental_historical_splits

equity_fundamental_historical_splits = get_equity_fundamental_historical_splits(symbol)
example_output = json.loads(equity_fundamental_historical_splits)
tools_examples["get_equity_fundamental_historical_splits"]=example_output
example_output

{'date': '1999-02-17',
 'numerator': 2.0,
 'denominator': 1.0,
 'split_ratio': None,
 'label': 'February 17, 99'}

In [43]:
agent_query(f"Provide latest split information for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LjLANC1zQ6eK1EuIcxSjGTPx', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_historical_splits'), type='function')]))
[ChatCompletionMessageToolCall(id='call_LjLANC1zQ6eK1EuIcxSjGTPx', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_historical_splits'), type='function')]
get_equity_fundamental_historical_splits({'symbol': 'MRK'})
{"date":"1999-02-17","numerator":2.0,"denominator":1.0,"split_ratio":null,"label":"February 17, 99"}
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The latest stock split for Merck & Co., Inc. (MRK) occurred on February 17, 1999. In this split, the stock was divided in a 2-for-1 ratio.', role='assistant', function_call=None, tool_calls=None))


The latest stock split for Merck & Co., Inc. (MRK) occurred on February 17, 1999. In this split, the stock was divided in a 2-for-1 ratio.

In [44]:
def get_balance_sheet_json(symbol):
    """Get the most recent balance sheet data for a given company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The balance sheet data in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.balance(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval

tools["get_balance_sheet_json"] = {
    "type": "function",
    "function": {
        "name": "get_balance_sheet_json",
        "description": "Given a stock symbol, get the most recent balance sheet data (detailed assets and liabilities) for a company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}

tools_dict["get_balance_sheet_json"] = get_balance_sheet_json

balance_sheet_json = get_balance_sheet_json(symbol)
example_output = json.loads(balance_sheet_json)
tools_examples["get_balance_sheet_json"]=example_output
example_output

{'period_ending': '2023-12-31',
 'fiscal_period': 'FY',
 'fiscal_year': 2023,
 'filing_date': '2024-04-11',
 'accepted_date': '2024-04-11T16:05:38',
 'reported_currency': 'USD',
 'cash_and_cash_equivalents': 6909000000.0,
 'short_term_investments': 252000000.0,
 'cash_and_short_term_investments': 7161000000.0,
 'net_receivables': 11072000000.0,
 'inventory': 6358000000.0,
 'other_current_assets': 7577000000.0,
 'total_current_assets': 32168000000.0,
 'plant_property_equipment_net': 24488000000.0,
 'goodwill': 21197000000.0,
 'intangible_assets': 18011000000.0,
 'goodwill_and_intangible_assets': 39208000000.0,
 'long_term_investments': 1584000000.0,
 'tax_assets': None,
 'other_non_current_assets': 9227000000.0,
 'non_current_assets': 74507000000.0,
 'other_assets': None,
 'total_assets': 106675000000.0,
 'accounts_payable': 3922000000.0,
 'short_term_debt': 1657000000.0,
 'tax_payables': 2649000000.0,
 'current_deferred_revenue': 4634000000.0,
 'other_current_liabilities': 15481000000.

In [45]:
agent_query(f"what are the latest total assets for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DNRYQnwZAlXof8wFtu8hkm8o', function=Function(arguments='{"symbol":"MRK"}', name='get_balance_sheet_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_DNRYQnwZAlXof8wFtu8hkm8o', function=Function(arguments='{"symbol":"MRK"}', name='get_balance_sheet_json'), type='function')]
get_balance_sheet_json({'symbol': 'MRK'})
{"period_ending":"2023-12-31","fiscal_period":"FY","fiscal_year":2023,"filing_date":"2024-04-11","accepted_date":"2024-04-11T16:05:38","reported_currency":"USD","cash_and_cash_equivalents":6909000000.0,"short_term_investments":252000000.0,"cash_and_short_term_investments":7161000000.0,"net_receivables":11072000000.0,"inventory":6358000000.0,"other_current_assets":7577000000.0,"total_current_assets":32168000000.0,"plant_property_equipment_net":24488000000.0,"goodwill":21

As of December 31, 2023, Merck & Co., Inc. (MRK) reported total assets amounting to $106.675 billion. This data is based on their latest available balance sheet filing as of April 11, 2024.

In [46]:
def get_balance_sheet_growth_json(symbol):
    """Given a stock symbol, get the percent changes in the most recent balance sheet data for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The JSON representation of the balance sheet growth.

    """
    retval = None
    retlist = obb.equity.fundamental.balance_growth(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval

tools["get_balance_sheet_growth_json"] = {
    "type": "function",
    "function": {
        "name": "get_balance_sheet_growth_json",
        "description": "Given a stock symbol, get the percent changes in the most recent balance sheet data for the company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}

tools_dict["get_balance_sheet_growth_json"] = get_balance_sheet_growth_json


balance_sheet_growth_json = get_balance_sheet_growth_json(symbol)
example_output = json.loads(balance_sheet_growth_json)
tools_examples["get_balance_sheet_growth_json"]=example_output
example_output

{'symbol': 'MRK',
 'date': '2023-12-31',
 'period': 'FY',
 'growth_cash_and_cash_equivalents': -0.45572711517252246,
 'growth_short_term_investments': -0.4939759036144578,
 'growth_cash_and_short_term_investments': -0.4571710127349909,
 'growth_net_receivables': 0.17164021164021165,
 'growth_inventory': 0.07562172221282355,
 'growth_other_current_assets': 0.056911703166410935,
 'growth_total_current_assets': -0.09949051004982924,
 'growth_property_plant_equipment_net': 0.14312389132667352,
 'growth_goodwill': -0.00033012639124693457,
 'growth_intangible_assets': -0.11140164783659776,
 'growth_goodwill_and_intangible_assets': -0.05461384515226774,
 'growth_long_term_investments': 0.5605911330049261,
 'growth_tax_assets': -1.0,
 'growth_other_non_current_assets': 0.19319798267166688,
 'growth_total_non_current_assets': 0.014556496636618644,
 'growth_other_assets': 0.0,
 'growth_total_assets': -0.022764748992304875,
 'growth_account_payables': -0.08020637898686679,
 'growth_short_term_deb

In [47]:
agent_query(f"what was the most recent percentage growth in total assets for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DbOmAYIruPqlmpGRogw50D17', function=Function(arguments='{"symbol":"MRK"}', name='get_balance_sheet_growth_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_DbOmAYIruPqlmpGRogw50D17', function=Function(arguments='{"symbol":"MRK"}', name='get_balance_sheet_growth_json'), type='function')]
get_balance_sheet_growth_json({'symbol': 'MRK'})
{"symbol":"MRK","date":"2023-12-31","period":"FY","growth_cash_and_cash_equivalents":-0.45572711517252246,"growth_short_term_investments":-0.4939759036144578,"growth_cash_and_short_term_investments":-0.4571710127349909,"growth_net_receivables":0.17164021164021165,"growth_inventory":0.07562172221282355,"growth_other_current_assets":0.056911703166410935,"growth_total_current_assets":-0.09949051004982924,"growth_property_plant_equipment_net":0.14312389

As of December 31, 2023, Merck & Co., Inc. (MRK) reported a decrease of 2.28% in total assets for the fiscal year.

In [48]:
def get_cash_flow_json(symbol):
    """Given a stock symbol, get the cash flow statement data for a given company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The cash flow statement data in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.cash(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval

tools["get_cash_flow_json"] = {
    "type": "function",
    "function": {
        "name": "get_cash_flow_json",
        "description": "Given a stock symbol, get the cash flow statement data for a given company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}

tools_dict["get_cash_flow_json"] = get_cash_flow_json


cash_flow_json = get_cash_flow_json(symbol)
example_output = json.loads(cash_flow_json)
tools_examples["get_cash_flow_json"]=example_output
example_output

{'period_ending': '2023-12-31',
 'fiscal_period': 'FY',
 'fiscal_year': 2023,
 'filing_date': '2024-04-11',
 'accepted_date': '2024-04-11T16:05:38',
 'reported_currency': 'USD',
 'net_income': 365000000.0,
 'depreciation_and_amortization': 3872000000.0,
 'deferred_income_tax': -1899000000.0,
 'stock_based_compensation': 645000000.0,
 'change_in_working_capital': -2205000000.0,
 'change_in_account_receivables': -1148000000.0,
 'change_in_inventory': -816000000.0,
 'change_in_account_payable': -380000000.0,
 'change_in_other_working_capital': 139000000.0,
 'change_in_other_non_cash_items': 12228000000.0,
 'net_cash_from_operating_activities': 13006000000.0,
 'purchase_of_property_plant_and_equipment': -3863000000.0,
 'acquisitions': -12032000000.0,
 'purchase_of_investment_securities': -955000000.0,
 'sale_and_maturity_of_investments': 1658000000.0,
 'other_investing_activities': 1109000000.0,
 'net_cash_from_investing_activities': -14083000000.0,
 'repayment_of_debt': -4184000000.0,
 'i

In [49]:
agent_query(f"what was the most recent cash flow from operations for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jivNnPhesaOKcMw9Q0hudH4a', function=Function(arguments='{"symbol":"MRK"}', name='get_cash_flow_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_jivNnPhesaOKcMw9Q0hudH4a', function=Function(arguments='{"symbol":"MRK"}', name='get_cash_flow_json'), type='function')]
get_cash_flow_json({'symbol': 'MRK'})
{"period_ending":"2023-12-31","fiscal_period":"FY","fiscal_year":2023,"filing_date":"2024-04-11","accepted_date":"2024-04-11T16:05:38","reported_currency":"USD","net_income":365000000.0,"depreciation_and_amortization":3872000000.0,"deferred_income_tax":-1899000000.0,"stock_based_compensation":645000000.0,"change_in_working_capital":-2205000000.0,"change_in_account_receivables":-1148000000.0,"change_in_inventory":-816000000.0,"change_in_account_payable":-380000000.0,"change_in_other

The most recent cash flow from operations for Merck & Co Inc (MRK) for the fiscal year ending December 31, 2023, was $13.006 billion.

In [50]:
def get_cash_flow_growth_json(symbol):
    """Given a stock symbol, get the changes in the company's cash flow statement items in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The JSON representation of the growth of the company's cash flow statement items in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.cash_growth(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval

tools["get_cash_flow_growth_json"] = {
    "type": "function",
    "function": {
        "name": "get_cash_flow_growth_json",
        "description": "Given a stock symbol, get the changes in the company's cash flow statement items.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}

tools_dict["get_cash_flow_growth_json"] = get_cash_flow_growth_json


cash_flow_growth_json = get_cash_flow_growth_json(symbol)
example_output = json.loads(cash_flow_growth_json)
tools_examples["get_cash_flow_growth_json"]=example_output
example_output

{'symbol': 'MRK',
 'date': '2023-12-31',
 'period': 'FY',
 'growth_net_income': -0.9748726421588875,
 'growth_depreciation_and_amortization': -0.009465336403172167,
 'growth_deferred_income_tax': -0.2110969387755102,
 'growth_stock_based_compensation': 0.1922365988909427,
 'growth_change_in_working_capital': 0.20740474478792237,
 'growth_accounts_receivables': -0.782608695652174,
 'growth_inventory': -4.068322981366459,
 'growth_accounts_payables': -0.314878892733564,
 'growth_other_working_capital': 1.082345971563981,
 'growth_other_non_cash_items': 1.73618259118371,
 'growth_net_cash_provided_by_operating_activities': -0.31887928777166796,
 'growth_investments_in_property_plant_and_equipment': 0.11964448495897903,
 'growth_acquisitions_net': -98.43801652892562,
 'growth_purchases_of_investments': 0.20681063122923588,
 'growth_sales_maturities_of_investments': 1.2995839112343968,
 'growth_other_investing_activities': 33.65625,
 'growth_net_cash_used_for_investing_activities': -1.83931

In [51]:
agent_query(f"what was the most recent change in cash flow from operations for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yhJm4BqDgPiXa50OH4vU3Pj3', function=Function(arguments='{"symbol":"MRK"}', name='get_cash_flow_growth_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_yhJm4BqDgPiXa50OH4vU3Pj3', function=Function(arguments='{"symbol":"MRK"}', name='get_cash_flow_growth_json'), type='function')]
get_cash_flow_growth_json({'symbol': 'MRK'})
{"symbol":"MRK","date":"2023-12-31","period":"FY","growth_net_income":-0.9748726421588875,"growth_depreciation_and_amortization":-0.009465336403172167,"growth_deferred_income_tax":-0.2110969387755102,"growth_stock_based_compensation":0.1922365988909427,"growth_change_in_working_capital":0.20740474478792237,"growth_accounts_receivables":-0.782608695652174,"growth_inventory":-4.068322981366459,"growth_accounts_payables":-0.314878892733564,"growth_other_working_ca

For Merck & Co. (MRK), the most recent change in cash flow from operations (operating cash flow) for the fiscal year ending December 31, 2023, was a decrease of approximately 31.89%.

In [52]:
def get_income_statement_json(symbol):
    """Given a stock symbol, get the latest income statement data for the company in JSON format

    Args:
        symbol (str): The stock symbol of the company.

    Returns:
        str: The income statement data in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.income(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


tools["get_income_statement_json"] = {
    "type": "function",
    "function": {
        "name": "get_income_statement_json",
        "description": "Given a stock symbol, get the latest income statement data for the company in JSON format",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}

tools_dict["get_income_statement_json"] = get_income_statement_json

income_json = get_income_statement_json(symbol)
example_output = json.loads(income_json)
tools_examples["get_income_statement_json"]=example_output
example_output

{'period_ending': '2023-12-31',
 'fiscal_period': 'FY',
 'fiscal_year': 2023,
 'filing_date': '2024-04-11',
 'accepted_date': '2024-04-11T16:05:38',
 'reported_currency': 'USD',
 'revenue': 59871000000.0,
 'cost_of_revenue': 15977000000.0,
 'gross_profit': 43894000000.0,
 'gross_profit_margin': 0.733142924,
 'general_and_admin_expense': 8082000000.0,
 'research_and_development_expense': 30530000000.0,
 'selling_and_marketing_expense': 2300000000.0,
 'selling_general_and_admin_expense': 10382000000.0,
 'other_expenses': 345000000.0,
 'total_operating_expenses': 40912000000.0,
 'cost_and_expenses': 56889000000.0,
 'interest_income': 365000000.0,
 'total_interest_expense': 1146000000.0,
 'depreciation_and_amortization': 3872000000.0,
 'ebitda': 6854000000.0,
 'ebitda_margin': 0.1144794642,
 'total_operating_income': 2982000000.0,
 'operating_income_margin': 0.0498070852,
 'total_other_income_expenses': -1093000000.0,
 'total_pre_tax_income': 1889000000.0,
 'pre_tax_income_margin': 0.03155

In [53]:
agent_query(f"what was the most recent net income for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xoApi92mmsbfjx1Dt01HqRHi', function=Function(arguments='{"symbol":"MRK"}', name='get_income_statement_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_xoApi92mmsbfjx1Dt01HqRHi', function=Function(arguments='{"symbol":"MRK"}', name='get_income_statement_json'), type='function')]
get_income_statement_json({'symbol': 'MRK'})
{"period_ending":"2023-12-31","fiscal_period":"FY","fiscal_year":2023,"filing_date":"2024-04-11","accepted_date":"2024-04-11T16:05:38","reported_currency":"USD","revenue":59871000000.0,"cost_of_revenue":15977000000.0,"gross_profit":43894000000.0,"gross_profit_margin":0.733142924,"general_and_admin_expense":8082000000.0,"research_and_development_expense":30530000000.0,"selling_and_marketing_expense":2300000000.0,"selling_general_and_admin_expense":10382000000.0,

The most recent net income reported by Merck & Co., Inc. (MRK) for the fiscal year ending December 31, 2023, was $365 million USD.

In [54]:
def get_income_growth_json(symbol):
    """Given a stock symbol, get the percent changes in the company's cash flow statement items in JSON format.

    Args:
        symbol (str): The stock symbol for the company.

    Returns:
        str: The JSON representation of the changes in the company's cash flow statement items.

    Raises:
        None

    Examples:
        >>> get_income_growth_json('AAPL')
        '{"item1": 100, "item2": 200, "item3": 300}'

    """
    retval = None
    retlist = obb.equity.fundamental.income_growth(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


tools["get_income_growth_json"] = {
    "type": "function",
    "function": {
        "name": "get_income_growth_json",
        "description": "Given a stock symbol, get the percent changes in the company's cash flow statement items in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_income_growth_json"] = get_income_growth_json

income_growth_json = get_income_growth_json(symbol)
example_output = json.loads(income_growth_json)
tools_examples["get_income_growth_json"] = example_output
example_output


{'symbol': 'MRK',
 'date': '2023-12-31',
 'period': 'FY',
 'growth_revenue': 0.00991852639036486,
 'growth_cost_of_revenue': -0.0823617253460456,
 'growth_gross_profit': 0.048290026748184944,
 'growth_gross_profit_ratio': 0.037994649555492255,
 'growth_research_and_development_expenses': 1.2534691467375259,
 'growth_general_and_administrative_expenses': 0.030604437643458302,
 'growth_selling_and_marketing_expenses': 0.045454545454545456,
 'growth_other_expenses': 1.2298467688207861,
 'growth_operating_expenses': 0.7342941924544298,
 'growth_cost_and_expenses': 0.3875027438355162,
 'growth_interest_expense': 0.19126819126819128,
 'growth_depreciation_and_amortization': -0.009465336403172167,
 'growth_ebitda': -0.6318814114614104,
 'growth_ebitda_ratio': -0.6354967465563471,
 'growth_operating_income': -0.7972807613868117,
 'growth_operating_income_ratio': -0.799271690572247,
 'growth_total_other_income_expenses_net': -1.630334486735871,
 'growth_income_before_tax': -0.8851252736560448,


In [55]:
agent_query(f"what was the most recent change in net income for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZEudnC63wKUfghFuQs294CUo', function=Function(arguments='{"symbol":"MRK"}', name='get_income_growth_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_ZEudnC63wKUfghFuQs294CUo', function=Function(arguments='{"symbol":"MRK"}', name='get_income_growth_json'), type='function')]
get_income_growth_json({'symbol': 'MRK'})
{"symbol":"MRK","date":"2023-12-31","period":"FY","growth_revenue":0.00991852639036486,"growth_cost_of_revenue":-0.0823617253460456,"growth_gross_profit":0.048290026748184944,"growth_gross_profit_ratio":0.037994649555492255,"growth_research_and_development_expenses":1.2534691467375259,"growth_general_and_administrative_expenses":0.030604437643458302,"growth_selling_and_marketing_expenses":0.045454545454545456,"growth_other_expenses":1.2298467688207861,"growth_operating_

For the fiscal year ending December 31, 2023, Merck & Co., Inc. (MRK) experienced a significant decrease in net income, showing a growth rate of approximately -97.49%.

In [56]:
def get_fundamental_metrics_json(symbol):
    """Given a stock symbol, get fundamental metrics for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The JSON representation of the fundamental metrics.

    """
    retval = None
    retlist = obb.equity.fundamental.metrics(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval



tools["get_fundamental_metrics_json"] = {
    "type": "function",
    "function": {
        "name": "get_fundamental_metrics_json",
        "description": "Given a stock symbol, get fundamental metrics for the company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_fundamental_metrics_json"] = get_fundamental_metrics_json

fundamental_metrics_json = get_fundamental_metrics_json(symbol)
example_output = json.loads(fundamental_metrics_json)
tools_examples["get_fundamental_metrics_json"] = example_output
example_output


{'symbol': 'MRK',
 'market_cap': 329420000000.0,
 'pe_ratio': 144.78,
 'foward_pe': 13.14,
 'eps': 0.9,
 'price_to_sales': 5.39,
 'price_to_book': 8.16,
 'book_value_per_share': 15.94,
 'price_to_cash': 57.68,
 'cash_per_share': 2.25,
 'price_to_free_cash_flow': 29.84,
 'debt_to_equity': 0.85,
 'long_term_debt_to_equity': 0.77,
 'quick_ratio': 0.99,
 'current_ratio': 1.25,
 'gross_margin': 0.7162999999999999,
 'profit_margin': 0.0377,
 'operating_margin': 0.0823,
 'return_on_assets': 0.0216,
 'return_on_investment': 0.0322,
 'return_on_equity': 0.0529,
 'payout_ratio': 20.656,
 'dividend_yield': None}

In [57]:
agent_query(f"what was the most recent PE ratio for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_v1tu33NXppj7jrOB8lQdqO9f', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_v1tu33NXppj7jrOB8lQdqO9f', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]
get_fundamental_metrics_json({'symbol': 'MRK'})
{"symbol":"MRK","market_cap":329420000000.0,"pe_ratio":144.78,"foward_pe":13.14,"eps":0.9,"price_to_sales":5.39,"price_to_book":8.16,"book_value_per_share":15.94,"price_to_cash":57.68,"cash_per_share":2.25,"price_to_free_cash_flow":29.84,"debt_to_equity":0.85,"long_term_debt_to_equity":0.77,"quick_ratio":0.99,"current_ratio":1.25,"gross_margin":0.7162999999999999,"profit_margin":0.0377,"operating_margin":0.0823,"return_on_assets":0.0216,"return_on_inve

As of the most recent data, the P/E (Price-to-Earnings) ratio for Merck & Co. Inc. (MRK) is 144.78.

In [58]:
def get_fundamental_ratios_json(symbol):
    """Given a stock symbol, get fundamental valuation ratios for the company in JSON format.

    Args:
        symbol (str): The stock symbol for the company.

    Returns:
        str: The fundamental valuation ratios for the company in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.ratios(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval



tools["get_fundamental_ratios_json"] = {
    "type": "function",
    "function": {
        "name": "get_fundamental_ratios_json",
        "description": "Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_fundamental_ratios_json"] = get_fundamental_ratios_json

fundamental_ratios = get_fundamental_ratios_json(symbol)
example_output = json.loads(fundamental_ratios)
tools_examples["get_fundamental_ratios_json"] = example_output
example_output


{'period_ending': '2023-12-31',
 'fiscal_period': 'FY',
 'fiscal_year': 2023,
 'current_ratio': 1.251965439402195,
 'quick_ratio': 0.7096209231727252,
 'cash_ratio': 0.26889546197555847,
 'days_of_sales_outstanding': 67.4997912177849,
 'days_of_inventory_outstanding': 145.25067284221066,
 'operating_cycle': 212.75046405999558,
 'days_of_payables_outstanding': 89.59942417224761,
 'cash_conversion_cycle': 123.15103988774797,
 'gross_profit_margin': 0.7331429239531659,
 'operating_profit_margin': 0.04980708523325149,
 'pretax_profit_margin': 0.031551168345275674,
 'net_profit_margin': 0.006096440680797047,
 'effective_tax_rate': 0.8004235044997353,
 'return_on_assets': 0.003421607686899461,
 'return_on_equity': 0.009712354647295175,
 'return_on_capital_employed': 0.03682345241476396,
 'net_income_per_ebt': 0.19322392800423505,
 'ebt_per_ebit': 0.6334674714956405,
 'ebit_per_revenue': 0.04980708523325149,
 'debt_ratio': 0.3312866182329505,
 'debt_equity_ratio': 0.9403688033846891,
 'long_t

In [59]:
agent_query(f"what was the most recent price to sales ratio for {symbol}", verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Z5uiBbZjT2yLjIBLRi0W2ele', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_Z5uiBbZjT2yLjIBLRi0W2ele', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]
get_fundamental_metrics_json({'symbol': 'MRK'})
{"symbol":"MRK","market_cap":329420000000.0,"pe_ratio":144.78,"foward_pe":13.14,"eps":0.9,"price_to_sales":5.39,"price_to_book":8.16,"book_value_per_share":15.94,"price_to_cash":57.68,"cash_per_share":2.25,"price_to_free_cash_flow":29.84,"debt_to_equity":0.85,"long_term_debt_to_equity":0.77,"quick_ratio":0.99,"current_ratio":1.25,"gross_margin":0.7162999999999999,"profit_margin":0.0377,"operating_margin":0.0823,"return_on_assets":0.0216,"return_on_inve

The most recent price to sales ratio for Merck & Co. Inc. (MRK) was 5.39.

In [60]:
def get_equity_fundamental_multiples(symbol):
    """Given a stock symbol, get fundamental equity valuation multiples for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The JSON representation of the valuation multiples.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.multiples(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval



tools["get_equity_fundamental_multiples"] = {
    "type": "function",
    "function": {
        "name": "get_equity_fundamental_multiples",
        "description": "Given a stock symbol, get fundamental equity valuation multiples for the company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_equity_fundamental_multiples"] = get_equity_fundamental_multiples


equity_fundamental_multiples = get_equity_fundamental_multiples(symbol)
example_output = json.loads(equity_fundamental_multiples)
tools_examples["get_equity_fundamental_multiples"] = example_output
example_output


{'symbol': 'MRK',
 'revenue_per_share_ttm': 24.25424397947098,
 'net_income_per_share_ttm': 0.9103829451243585,
 'operating_cash_flow_per_share_ttm': 5.8258981444926965,
 'free_cash_flow_per_share_ttm': 4.358468219502566,
 'cash_per_share_ttm': 2.2546387682589812,
 'book_value_per_share_ttm': 15.935254638768258,
 'tangible_book_value_per_share_ttm': 0.6596920647453612,
 'shareholders_equity_per_share_ttm': 15.935254638768258,
 'interest_debt_per_share_ttm': 13.985787603632057,
 'market_cap_ttm': 329417268600.0,
 'enterprise_value_ttm': 357965268600.0,
 'pe_ratio_ttm': 142.86295750216826,
 'price_to_sales_ratio_ttm': 5.361958275278338,
 'pocf_ratio_ttm': 22.32445483499356,
 'pfcf_ratio_ttm': 29.838520706521738,
 'pb_ratio_ttm': 8.161777326330395,
 'ptb_ratio_ttm': 8.161777326330395,
 'ev_to_sales_ttm': 5.82663696529722,
 'enterprise_value_over_ebitda_ttm': 41.06519084547436,
 'ev_to_operating_cash_flow_ttm': 24.257319821101852,
 'ev_to_free_cash_flow_ttm': 32.42439027173913,
 'earnings_

In [61]:
agent_query(f"what was the most recent revenue per share for {symbol}", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vpP55XqZMEetDLksXHo3god2', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_multiples'), type='function')]))
[ChatCompletionMessageToolCall(id='call_vpP55XqZMEetDLksXHo3god2', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_multiples'), type='function')]
get_equity_fundamental_multiples({'symbol': 'MRK'})
{"symbol":"MRK","revenue_per_share_ttm":24.25424397947098,"net_income_per_share_ttm":0.9103829451243585,"operating_cash_flow_per_share_ttm":5.8258981444926965,"free_cash_flow_per_share_ttm":4.358468219502566,"cash_per_share_ttm":2.2546387682589812,"book_value_per_share_ttm":15.935254638768258,"tangible_book_value_per_share_ttm":0.6596920647453612,"shareholders_equity_per_share_ttm":15.935254638768258,"interest_debt_per_share_ttm":13.9857876

The most recent revenue per share for Merck & Co., Inc. (MRK) on a trailing twelve-month (TTM) basis is $24.254 per share.

In [62]:
def get_historical_eps(symbol):
    """Given a stock symbol, get historical earnings per share data for the company in JSON format.
    """
    retval = None
    try:
        obj = obb.equity.fundamental.historical_eps(symbol)
        retval = obj.to_dataframe().to_json(orient='records')
    except Exception as exc:
        print(exc)
    return retval


tools["get_historical_eps"] = {
    "type": "function",
    "function": {
        "name": "get_historical_eps",
        "description": "Given a stock symbol, get historical earnings per share data for the company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_historical_eps"] = get_historical_eps


fundamental_historical_eps = get_historical_eps(symbol)
example_output = json.loads(fundamental_historical_eps)
tools_examples["get_historical_eps"] = example_output
example_output


[{'symbol': 'MRK',
  'eps_actual': 0.35,
  'eps_estimated': 0.35,
  'surprise': None,
  'surprise_percent': None,
  'reported_date': 829699200000,
  'reportTime': 'pre-market'},
 {'symbol': 'MRK',
  'eps_actual': 0.4,
  'eps_estimated': 0.4,
  'surprise': None,
  'surprise_percent': None,
  'reported_date': 837648000000,
  'reportTime': 'pre-market'},
 {'symbol': 'MRK',
  'eps_actual': 0.42,
  'eps_estimated': 0.41,
  'surprise': 0.01,
  'surprise_percent': 0.02439,
  'reported_date': 845424000000,
  'reportTime': 'pre-market'},
 {'symbol': 'MRK',
  'eps_actual': 0.44,
  'eps_estimated': 0.43,
  'surprise': 0.01,
  'surprise_percent': 0.023256,
  'reported_date': 854409600000,
  'reportTime': 'pre-market'},
 {'symbol': 'MRK',
  'eps_actual': 0.42,
  'eps_estimated': 0.42,
  'surprise': None,
  'surprise_percent': None,
  'reported_date': 861148800000,
  'reportTime': 'pre-market'},
 {'symbol': 'MRK',
  'eps_actual': 0.48,
  'eps_estimated': 0.48,
  'surprise': None,
  'surprise_percent

In [63]:
pd.DataFrame(example_output)

,symbol,eps_actual,eps_estimated,surprise,surprise_percent,reported_date,reportTime
0,MRK,0.35,0.35,NaN,NaN,829699200000,pre-market
1,MRK,0.40,0.40,NaN,NaN,837648000000,pre-market
2,MRK,0.42,0.41,0.01,0.024390,845424000000,pre-market
3,MRK,0.44,0.43,0.01,0.023256,854409600000,pre-market
4,MRK,0.42,0.42,NaN,NaN,861148800000,pre-market
...,...,...,...,...,...,...,...
107,MRK,1.62,1.54,0.08,0.051948,1675296000000,pre-market
108,MRK,1.40,1.32,0.08,0.060606,1682553600000,pre-market
109,MRK,-2.06,-2.18,0.12,0.055046,1690848000000,pre-market
110,MRK,2.13,1.95,0.18,0.092308,1698278400000,pre-market


In [64]:
agent_query(f"what was the actual EPS for {symbol} in the quarter ended 2022-09-30?")


The actual earnings per share (EPS) for Merck & Co., Inc. (MRK) for the quarter ended September 30, 2022, was $1.62.


In [65]:
def get_equity_fundamental_dividend(symbol):
    """Given a stock symbol, get the latest dividend data for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The JSON representation of the last dividend data.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.dividends(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval


tools["get_equity_fundamental_dividend"] = {
    "type": "function",
    "function": {
        "name": "get_equity_fundamental_dividend",
        "description": "Given a stock symbol, get the latest dividend data for the company in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_equity_fundamental_dividend"] = get_equity_fundamental_dividend

equity_fundamental_dividends=get_equity_fundamental_dividend(symbol)
example_output = json.loads(equity_fundamental_dividends)
tools_examples["get_equity_fundamental_dividend"] = example_output
example_output


{'ex_dividend_date': '2024-03-14',
 'amount': 0.77,
 'label': 'March 14, 24',
 'adj_dividend': 0.77,
 'record_date': '2024-03-15',
 'payment_date': '2024-04-05',
 'declaration_date': '2024-01-23'}

In [66]:
agent_query(f"what was the latest dividend data for {symbol}?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wTJ8QbiTr7GIrGGN7yYZgBNh', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_dividend'), type='function')]))
[ChatCompletionMessageToolCall(id='call_wTJ8QbiTr7GIrGGN7yYZgBNh', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_dividend'), type='function')]
get_equity_fundamental_dividend({'symbol': 'MRK'})
{"ex_dividend_date":"2024-03-14","amount":0.77,"label":"March 14, 24","adj_dividend":0.77,"record_date":"2024-03-15","payment_date":"2024-04-05","declaration_date":"2024-01-23"}
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The latest dividend data for Merck & Co., Inc. (MRK) is as follows:\n- **Ex-Dividend Date**: March 14, 2024\n- **Amount**: $0.77 per share\n- **Adjusted Dividend**: $0.77\n- **R

The latest dividend data for Merck & Co., Inc. (MRK) is as follows:
- **Ex-Dividend Date**: March 14, 2024
- **Amount**: $0.77 per share
- **Adjusted Dividend**: $0.77
- **Record Date**: March 15, 2024
- **Payment Date**: April 5, 2024
- **Declaration Date**: January 23, 2024

In [67]:
def get_trailing_dividend_yield_json(symbol):
    """Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        dict: A dictionary containing the 1 year trailing dividend yield for the given company over time.

    Raises:
        Exception: If there is an error retrieving the dividend yield.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.trailing_dividend_yield(symbol=symbol, year=2023)
        retval = obj.to_dataframe().to_json(orient='records')
    except Exception as exc:
        print(exc)
    return retval


tools["get_trailing_dividend_yield_json"] = {
    "type": "function",
    "function": {
        "name": "get_trailing_dividend_yield_json",
        "description": "Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_trailing_dividend_yield_json"] = get_trailing_dividend_yield_json

trailing_dividend_yield=get_trailing_dividend_yield_json(symbol)
example_output = json.loads(trailing_dividend_yield)
tools_examples["get_trailing_dividend_yield_json"] = example_output
example_output


[{'trailing_dividend_yield': 0.0241599319},
 {'trailing_dividend_yield': 0.0242444938},
 {'trailing_dividend_yield': 0.0244049154},
 {'trailing_dividend_yield': 0.0244658856},
 {'trailing_dividend_yield': 0.0247472987},
 {'trailing_dividend_yield': 0.0249122807},
 {'trailing_dividend_yield': 0.0245908737},
 {'trailing_dividend_yield': 0.0248056599},
 {'trailing_dividend_yield': 0.0250728348},
 {'trailing_dividend_yield': 0.025},
 {'trailing_dividend_yield': 0.0252894034},
 {'trailing_dividend_yield': 0.0255694607},
 {'trailing_dividend_yield': 0.0260144728},
 {'trailing_dividend_yield': 0.0257223078},
 {'trailing_dividend_yield': 0.0256017308},
 {'trailing_dividend_yield': 0.0252399573},
 {'trailing_dividend_yield': 0.0251083017},
 {'trailing_dividend_yield': 0.0258158349},
 {'trailing_dividend_yield': 0.0261486051},
 {'trailing_dividend_yield': 0.0257432922},
 {'trailing_dividend_yield': 0.0256526059},
 {'trailing_dividend_yield': 0.0258510832},
 {'trailing_dividend_yield': 0.02582052

In [68]:
agent_query(f"what was the dividend yield for {symbol} as of the end of 2023?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zxqb3xYc7KjTFkQzxQEGKFzy', function=Function(arguments='{"symbol":"MRK"}', name='get_trailing_dividend_yield_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_zxqb3xYc7KjTFkQzxQEGKFzy', function=Function(arguments='{"symbol":"MRK"}', name='get_trailing_dividend_yield_json'), type='function')]
get_trailing_dividend_yield_json({'symbol': 'MRK'})
[{"trailing_dividend_yield":0.0241599319},{"trailing_dividend_yield":0.0242444938},{"trailing_dividend_yield":0.0244049154},{"trailing_dividend_yield":0.0244658856},{"trailing_dividend_yield":0.0247472987},{"trailing_dividend_yield":0.0249122807},{"trailing_dividend_yield":0.0245908737},{"trailing_dividend_yield":0.0248056599},{"trailing_dividend_yield":0.0250728348},{"trailing_dividend_yield":0.025},{"trailing_dividend_yield":0.0252894034}

As of the end of 2023, the trailing dividend yield for Merck & Co., Inc. (MRK) was approximately 2.31%.

In [69]:
TODAY = datetime.today().strftime('%Y-%m-%d')
YESTERDAY = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

TODAY, YESTERDAY


('2024-05-12', '2024-05-11')

In [70]:
def get_price_historical_json(symbol, start_date=YESTERDAY, end_date=TODAY):
    """
    Retrieves the historical price data for a given symbol within a specified date range in JSON format.

    Args:
        symbol (str): The symbol of the equity.
        start_date (str, optional): The start date of the historical data. Defaults to YESTERDAY.
        end_date (str, optional): The end date of the historical data. Defaults to TODAY.

    Returns:
        dict: The historical price data in JSON format.

    """
    retval = None
    try:
        obj = obb.equity.price.historical(symbol=symbol, start_date=start_date, end_date=end_date)
        retval = obj.json()
    except Exception as exc:
        print(exc)
    return retval



tools["get_price_historical_json"] = {
    "type": "function",
    "function": {
        "name": "get_price_historical_json",
        "description": "Retrieves the historical price data for a given symbol within a specified date range in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_price_historical_json"] = get_price_historical_json

price_historical = get_price_historical_json(symbol)
example_output = json.loads(price_historical)
tools_examples["get_price_historical_json"] = example_output
example_output


{'id': '0664112b-2d5b-7003-8000-e948c1b4439e',
 'results': [],
 'provider': 'alpha_vantage',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'alpha_vantage'},
    'standard_params': {'symbol': 'MRK',
     'interval': '1d',
     'start_date': '2024-05-11',
     'end_date': '2024-05-12'},
    'extra_params': {}},
   'duration': 107482291,
   'route': '/equity/price/historical',
   'timestamp': '2024-05-12T15:04:18.729086'}}}

In [71]:
def get_price_performance_json(symbol):
    """Given a stock symbol, get price performance data for the stock for different time periods in JSON format."""
    retval = None
    try:
        obj = obb.equity.price.performance(symbol=symbol)
        retval = obj.json()
    except Exception as exc:
        print(exc)
    return retval


tools["get_price_performance_json"] = {
    "type": "function",
    "function": {
        "name": "get_price_performance_json",
        "description": "Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_price_performance_json"] = get_price_performance_json


price_performance = get_price_performance_json(symbol)
example_output = json.loads(price_performance)
tools_examples["get_price_performance_json"] = example_output
example_output


{'id': '0664112b-2f4f-71b2-8000-74cd5ae6a819',
 'results': [{'symbol': 'MRK',
   'one_day': -0.0013,
   'wtd': None,
   'one_week': 0.02,
   'mtd': None,
   'one_month': 0.031,
   'qtd': None,
   'three_month': 0.036699999999999997,
   'six_month': 0.2458,
   'ytd': 0.193,
   'one_year': 0.10310000000000001,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.013999999999999999,
   'volatility_month': 0.0148,
   'price': 130.06,
   'volume': 5722074.0,
   'average_volume': 8210000.000000001,
   'relative_volume': 0.7,
   'analyst_recommendation': None,
   'analyst_score': 1.52}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 109126833,
   'route': '/equity/price/performance',
   'timestamp': '2024-05-12T15:04:18.84815

In [74]:
agent_query(f"what was the performance for {symbol} from 1 year ago?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yhJm4BqDgPiXa50OH4vU3Pj3', function=Function(arguments='{"symbol":"MRK"}', name='get_price_performance_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_yhJm4BqDgPiXa50OH4vU3Pj3', function=Function(arguments='{"symbol":"MRK"}', name='get_price_performance_json'), type='function')]
get_price_performance_json({'symbol': 'MRK'})
{"id":"06641130-1349-7385-8000-7984d272a420","results":[{"symbol":"MRK","one_day":-0.0013,"wtd":null,"one_week":0.02,"mtd":null,"one_month":0.031,"qtd":null,"three_month":0.036699999999999997,"six_month":0.2458,"ytd":0.193,"one_year":0.10310000000000001,"two_year":null,"three_year":null,"four_year":null,"five_year":null,"ten_year":null,"max":null,"volatility_week":0.013999999999999999,"volatility_month":0.0148,"price":130.06,"volume":5722074.0,"average_volum

Over the past year, Merck & Co. Inc. (MRK) has shown a performance increase of approximately 10.31%.

In [75]:
def get_etf_equity_exposure_json(symbol):
    """Given a stock symbol, get the ETFs with exposure to the stock in JSON format.

    Args:
        symbol (str): The symbol of the stock.

    Returns:
        dict: A dictionary containing the ETFs with exposure to the specified stock.

    Raises:
        Exception: If an error occurs while retrieving the ETFs.

    """
    retval = None
    try:
        obj = obb.etf.equity_exposure(symbol)
        retval = obj.to_dataframe().to_json(orient='records')
    except Exception as exc:
        print(exc)
    return retval


tools["get_etf_equity_exposure_json"] = {
    "type": "function",
    "function": {
        "name": "get_etf_equity_exposure_json",
        "description": "Given a stock symbol, get the ETFs with exposure to the stock in JSON format.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol."
                }
            },
            "required": ["symbol"]
        }
    }
}
tools_dict["get_etf_equity_exposure_json"] = get_etf_equity_exposure_json

equity_exposure = get_etf_equity_exposure_json(symbol)
example_output = json.loads(equity_exposure)
tools_examples["get_etf_equity_exposure_json"] = example_output
example_output


[{'equity_symbol': 'MRK',
  'etf_symbol': 'VTSAX',
  'shares': 79663326.0,
  'weight': 0.0066,
  'market_value': 10511575865.7},
 {'equity_symbol': 'MRK',
  'etf_symbol': 'VTI',
  'shares': 79663326.0,
  'weight': 0.0066,
  'market_value': 10511575865.7},
 {'equity_symbol': 'MRK',
  'etf_symbol': 'VSMPX',
  'shares': 79663326.0,
  'weight': 0.0066,
  'market_value': 10511575865.7},
 {'equity_symbol': 'MRK',
  'etf_symbol': 'VITSX',
  'shares': 79663326.0,
  'weight': 0.0066,
  'market_value': 10511575865.7},
 {'equity_symbol': 'MRK',
  'etf_symbol': 'VOO',
  'shares': 63867826.0,
  'weight': 0.0076,
  'market_value': 8427359640.7},
 {'equity_symbol': 'MRK',
  'etf_symbol': 'VFINX',
  'shares': 63867826.0,
  'weight': 0.0076,
  'market_value': 8427359640.7},
 {'equity_symbol': 'MRK',
  'etf_symbol': 'VFIAX',
  'shares': 63867826.0,
  'weight': 0.0076,
  'market_value': 8427359640.7},
 {'equity_symbol': 'MRK',
  'etf_symbol': 'SPY',
  'shares': 29743469.0,
  'weight': 0.00753024,
  'mark

In [ ]:
agent_query(f"which etf has the highest weight in {symbol} ?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7OtJNg8VOqMoMXDelhNiodXh', function=Function(arguments='{"symbol":"MRK"}', name='get_etf_equity_exposure_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_7OtJNg8VOqMoMXDelhNiodXh', function=Function(arguments='{"symbol":"MRK"}', name='get_etf_equity_exposure_json'), type='function')]
get_etf_equity_exposure_json({'symbol': 'MRK'})
[{"equity_symbol":"MRK","etf_symbol":"VTSAX","shares":79663326.0,"weight":0.0066,"market_value":10511575865.7000007629},{"equity_symbol":"MRK","etf_symbol":"VTI","shares":79663326.0,"weight":0.0066,"market_value":10511575865.7000007629},{"equity_symbol":"MRK","etf_symbol":"VSMPX","shares":79663326.0,"weight":0.0066,"market_value":10511575865.7000007629},{"equity_symbol":"MRK","etf_symbol":"VITSX","shares":79663326.0,"weight":0.0066,"market_value":10511